In [1]:
sponge_path = r"D:\Fourth-Year\sponge-database"

In [ ]:
# new preprocessing with the goal to
# remove trailing and leading zeros
# yyyy-mm-dd

from pathlib import Path
import pandas as pd
spongeSample_patj = Path.cwd()

def date_iso(df):
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        return df

def main():
    csv_path = '50_samples.csv'
    df = pd.read_csv(csv_path)
    df = date_iso(df)  # <-- assign the modified DataFrame back
    output = '50_samples_final.csv'
    df.to_csv(output, index=False, quoting=1, encoding='utf-8')

        
if __name__ == "__main__":
    main()

TypeError: date_iso() missing 1 required positional argument: 'df'

In [102]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path

sponge_path = Path.cwd()

def general_cleaning(df):
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df.columns = [re.sub(r'\s+', '_', col.strip().lower()) for col in df.columns]
    df = df.replace({r'"': '', r"'": '', r'\\': ''}, regex=True)
    df = df.replace({r'\r\n|\r': '\n'}, regex=True)
    return df

def data_consistency(df):
    for col in ['dive_no', 'otu']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')


    df = df.replace({'yes': True, 'no': False, 'Yes': True, 'No': False}, regex=True)
    return df


def date_cleaning(df):
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')
    return df

def categorical_normalization(df):
    categorical_columns = ['location', 'site', 'sample_code']
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.lower().str.replace('/', '-', regex=False)
    return df

import numpy as np
import pandas as pd

def fill_missing_values(df):
    # Convert empty strings to NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)

    # Fill string (object) columns with 'unknown'
    string_cols = df.select_dtypes(include='object').columns
    df[string_cols] = df[string_cols].fillna('unknown')

    # For numeric columns, keep NaN as is or fill with sentinel if you want
    numeric_cols = df.select_dtypes(include=['number']).columns
    # Here we just keep NaN to show missing numeric data clearly
    df[numeric_cols] = df[numeric_cols].where(pd.notnull(df[numeric_cols]), np.nan)

    return df


# def fill_unknown_values(df, columns):
#     for col in columns:
#         if col in df.columns:
#             df[col] = df[col].fillna('unknown')
#     return df

def main():
    csv_path = sponge_path / "PGC_MCE2_Sponge Diversity 2023 - Raw Data.csv"
    df = pd.read_csv(csv_path)

    df = general_cleaning(df)
    df = data_consistency(df)
    df = categorical_normalization(df)
    df = fill_missing_values(df)
    df = date_cleaning(df)
    # df = fill_unknown_values(df, ['putative_id', 'barcode_sequences', 'skeletal'])

    for col in ['barcode_sequences', 'skeletal']:
        if col in df.columns:df[col] = df[col].astype('string').fillna('unknown')

    output_path = sponge_path / "PGC_Raw_Data_Cleaned.csv"
    df.to_csv(output_path, index=False, quoting=1, encoding='utf-8')

    print(f"Cleaned data saved to: {output_path}")

if __name__ == "__main__":
    main()


Cleaned data saved to: c:\Users\User\Desktop\Sponge-Database\PGC_Raw_Data_Cleaned.csv


In [93]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

sponge_path = Path.cwd()

def general_cleaning(df):
    # Normalize column names (lowercase and underscores)
    df.columns = [re.sub(r'\s+', '_', col.strip().lower()) for col in df.columns]

    # Rename functional form column
    if 'functional_form_(schonberg,_2021)' in df.columns:
        df.rename(columns={'functional_form_(schonberg,_2021)': 'functional_form'}, inplace=True)

    # Strip whitespace and remove quotes/backslashes in all except photo
    cols_to_clean = [c for c in df.columns if c != 'photo']
    df[cols_to_clean] = df[cols_to_clean].applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df[cols_to_clean] = df[cols_to_clean].replace({r'"': '', r"'": '', r'\\': ''}, regex=True)
    df[cols_to_clean] = df[cols_to_clean].replace({r'\r\n|\r': '\n'}, regex=True)

    return df

def data_consistency(df):
    # Convert numeric columns
    for col in ['count_no', 'otu_id']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Only convert yes/no in explicitly boolean columns if you have any
    # (Avoid replacing 'FALSE' in categorical text)
    # If you know which columns are boolean, replace only there.

    return df

def categorical_normalization(df):
    # Lowercase relevant categorical columns and replace '/' with '-'
    for col in ['functional_form', 'growth_form', 'color', 'surface_texture', 
                'oscule_shape', 'oscule_distribution', 'ostia', 'putative_id']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.lower().str.replace('/', '-', regex=False)

    nan_pattern = r'^\s*(nan)?\s*$'

    # For surface_texture
    if 'surface_texture' in df.columns:
        # Replace empty or 'nan' strings with np.nan (case-insensitive)
        mask = df['surface_texture'].str.match(nan_pattern, flags=re.IGNORECASE, na=False)
        df.loc[mask, 'surface_texture'] = np.nan
        df['surface_texture'] = df['surface_texture'].fillna('unknown')

    # For putative_id
    if 'putative_id' in df.columns:
        mask = df['putative_id'].str.match(nan_pattern, flags=re.IGNORECASE, na=False)
        df.loc[mask, 'putative_id'] = np.nan
        df['putative_id'] = df['putative_id'].fillna('unknown')

    return df

def fill_missing_values(df):
    # Replace empty strings with NaN for all string columns except photo
    string_cols = [c for c in df.select_dtypes(include='object').columns if c != 'photo']
    df[string_cols] = df[string_cols].replace(r'^\s*$', np.nan, regex=True)

    # Fill missing string values with 'unknown' except photo and surface_texture & putative_id (already handled)
    for col in string_cols:
        if col not in ['surface_texture', 'putative_id']:
            df[col] = df[col].fillna('unknown')

    return df

def main():
    csv_path = sponge_path / "PGC_MCE2_Sponge Diversity 2023 - OTUs.csv"
    output_path = sponge_path / "PGC_OTU_Data_Cleaned.csv"

    if not csv_path.exists():
        print(f"ERROR: File not found at {csv_path}")
        return

    df = pd.read_csv(csv_path)
    df = general_cleaning(df)
    df = data_consistency(df)
    df = categorical_normalization(df)
    df = fill_missing_values(df)

    df.to_csv(output_path, index=False, quoting=1, encoding='utf-8')

    print(f"✅ Cleaned data saved to: {output_path}")
    print("📄 Columns:", df.columns.tolist())

if __name__ == "__main__":
    main()


✅ Cleaned data saved to: c:\Users\User\Desktop\Sponge-Database\PGC_OTU_Data_Cleaned.csv
📄 Columns: ['otu_id', 'photo', 'functional_form', 'growth_form', 'color', 'surface_texture', 'oscule_shape', 'oscule_distribution', 'ostia', 'count_no', 'putative_id']


C:\Users\User\AppData\Local\Temp\ipykernel_11504\822125561.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_to_clean] = df[cols_to_clean].applymap(lambda x: x.strip() if isinstance(x, str) else x)
